<a href="https://colab.research.google.com/github/MavClarito/Emerging-Tech-1/blob/main/Clarito_CPE018_MidtermExam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CPE018 Midterm Exam (1st Sem, A.Y. 2023-2024)


Student Submission Details:
* Name: Vincent Maverick D. Clarito
* Section: CPE31S2
* Schedule: 11/17/23
* Instructor: Engr. Roman M. Richard

---

## Intended Learning Outcomes

By the end of this activity, the student should be able to:
* ILO1: Demonstrate different methods for feature matching and detection learned in class and indepdentently from new sources.
* ILO2: Evaluate the accuracy of different feature matching and detection methods and scrutinize its applicability in solving a given real-life problem.

---

## Tasks

For this examination, you must create a **mood detection** program with an object-oriented programming approach (same as project CAMEO), it must detect mood changes through the use of algorithms/techniques/schemes learned in class, and from external sources.

In this file, you have to include for each section of your solution your completion of the following:

* Part 1: **Face Detection**: Once your face is detected using any algorithm, it must draw an ROI. The color for the ROI is your choice; however, it must detect for all faces in the frame and draw a corresponding ROI.
* Part 2: **Face Recognition**: The detected face must then be recognized, using any of the provided tools in class, the ROIs must indicate whether it is your face or someone it doesn't recognize.
* Part 3: **Mood Detection**: Use three different feature detection and matching techniques to determine three emotion: happy, sad and neutral. Two of the techniques must be learned from class, and 1 must be one you independently learned.

Properly show through your notebook the output for each part of the exam.

---

## Procedure and Outputs

Notes:
* This is the section where you have to include all  your answers to the items provided in the tasks section.
* Tasks 1 and 2 contribute directly to ILO1: Demonstrate different methods for feature matching and detection learned in class and indepdentently from new sources.
* Task 3 contributes directly to ILO2: Evaluate the accuracy of different feature matching and detection methods and scrutinize its applicability in solving a given real-life problem.

### Task 1: Face Detection

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Nov 16 11:37:02 2023

@author: Mav
"""

import cv2
import threading

#I used a multithreading where it seperates the video capture and processes
# in order for my computer to have less lag.
class VideoStream:
    def __init__(self, src=0):
        self.stream = cv2.VideoCapture(src)
        (self.grabbed, self.frame) = self.stream.read()
        self.stopped = False

    def start(self):
        threading.Thread(target=self.update, args=()).start()
        return self

    def update(self):
        while not self.stopped:
            (self.grabbed, self.frame) = self.stream.read()

    def read(self):
        return self.frame

    def stop(self):
        self.stopped = True

def detect():
    face_cascade = cv2.CascadeClassifier(
   'C:/Users/Mav/Documents/Midterm Exam - Emerging Tech/haarcascade_frontalface_default.xml')

    vs = VideoStream().start()

    while True:
        frame = vs.read()
        small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        gray = cv2.cvtColor(small_frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
 #Scalability of the x,y coordinates as well as the width and height of the drawn rectangle
            x *= 2
            y *= 2
            w *= 2
            h *= 2
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            roi_gray = gray[y:y+h, x:x+w]

        cv2.imshow("camera", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    vs.stop()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    detect()

In [ ]:
#Reference: https://stackoverflow.com/questions/67567464/multi-threading-in-image-processing-video-python-opencv

### Task 2: Face Recognition

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Nov 16 11:45:07 2023

@author: Mav
"""

import numpy as np
import os
import sys
import cv2

haarcascade = 'C:/Users/Mav/Documents/Midterm Exam - Emerging Tech/haarcascade_frontalface_default.xml'
path = 'C:/Users/Mav/Documents/Midterm Exam - Emerging Tech/Mav-FR/Mav-Resized'


def read_images(path, sz=None):
    c = 0
    X, y = [], []

    for dirname, dirnames, filenames in os.walk(path):
        for subdirname in dirnames:
            subject_path = os.path.join(dirname, subdirname)
            for filename in os.listdir(subject_path):
                try:
                    if filename == ".directory":
                        continue
                    filepath = os.path.join(subject_path, filename)
                    im = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

                    if sz is not None:
                        im = cv2.resize(im, (200, 200))

                    X.append(np.asarray(im, dtype=np.uint8))
                    y.append(c)

                except IOError as e:
                    print(f"I/O Error({e.errno}): {e.strerror}")
                except:
                    print("Unexpected error:", sys.exc_info()[0])
                    raise
            c = c + 1
    return [X, y]

def face_rec():
    names = ['Unrecognized', 'Mav', 'MarkZ']

    [X, y] = read_images(path)
    y = np.asarray(y, dtype=np.int32)

    model = cv2.face.EigenFaceRecognizer_create()
    model.train(X, y)

    camera = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier(haarcascade)

    while True:
        ret, img = camera.read()
        if not ret:
            print("Error reading frame.")
            break

        faces = face_cascade.detectMultiScale(img, 1.3, 5)

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            gray = cv2.cvtColor(img[y:y + h, x:x + w], cv2.COLOR_BGR2GRAY)
            roi = cv2.resize(gray, (200, 200), interpolation=cv2.INTER_LINEAR)

            try:
                params = model.predict(roi)
                label = names[params[0]]
                cv2.putText(img, label + ", " + str(params[1]), (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            except:
                continue

        cv2.imshow("camera", img)
        if cv2.waitKey(30) & 0xFF == ord("q"):
            break

    camera.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    face_rec()

### Task 3: Mood Detection

In [ ]:
# BRIEF Algorithm w/ Face recognition and Detection (1st Technique)
# -*- coding: utf-8 -*-
"""
Created on Thu Nov 16 20:35:08 2023

@author: Mav
"""
import cv2
import numpy as np
import os

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
    return images

happy_images = load_images_from_folder("C:/Users/Mav/Documents/Midterm Exam - Emerging Tech/Mav-MD/Happy/pgm resized")
neutral_images = load_images_from_folder("C:/Users/Mav/Documents/Midterm Exam - Emerging Tech/Mav-MD/Neutral/pgm resized")
sad_images = load_images_from_folder("C:/Users/Mav/Documents/Midterm Exam - Emerging Tech/Mav-MD/Sad/pgm resized")

print("Number of Happy Images:", len(happy_images))
print("Number of Neutral Images:", len(neutral_images))
print("Number of Sad Images:", len(sad_images))

brisk = cv2.BRISK_create()
camera = cv2.VideoCapture(0)

while True:
    ret, frame = camera.read()
    if not ret:
        print("Error reading frame.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    print("Number of Faces Detected:", len(faces))

    for (x, y, w, h) in faces:
        roi = gray[y:y + h, x:x + w]
        keypoints, descriptors = brisk.detectAndCompute(roi, None)

        if descriptors is not None:
            descriptors = descriptors.astype(np.float32)
        else:
            continue  # Skip the current face

        match_scores = []
        for mood_images, mood_label in zip([happy_images, neutral_images, sad_images],
                                           ["Happy", "Neutral", "Sad"]):
            mood_match = 0

            for img in mood_images:
                kp_img, des_img = brisk.detectAndCompute(img, None)

                if des_img is None:
                    continue

                des_img = des_img.astype(np.float32)
                bf = cv2.BFMatcher()
                matches = bf.knnMatch(descriptors, des_img, k=2)
                good_matches = []
                for m, n in matches:
                    if m.distance < 0.75 * n.distance:
                        good_matches.append(m)

                mood_match += len(good_matches)

            match_scores.append((mood_match, mood_label))

        #This chooses the mood with the highest match or that matched my camera
        best_match = max(match_scores, key=lambda x: x[0])
        mood = best_match[1]

        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(frame, mood, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()

In [ ]:
#Reference: https://pypi.org/project/face-recognition/l;o

In [ ]:
# Use of Haarcascade.smile for mood detection (2nd Technique)

# -*- coding: utf-8 -*-
"""
Created on Thu Nov 16 19:35:42 2023

@author: Mav
"""
#Technique 2 using smile haarcascade
import cv2

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

camera = cv2.VideoCapture(0)

while True:
    ret, frame = camera.read()
    if not ret:
        print("Error reading frame.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    print("Number of Faces Detected:", len(faces))

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]

        smiles = smile_cascade.detectMultiScale(roi_gray, scaleFactor=1.8, minNeighbors=20)

        # Determine mood based on smile detection
        if len(smiles) > 0:
            mood = "Happy"
        else:
            mood = "Neutral"

        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(frame, mood, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()



In [ ]:
# Mood detection with deep learning (Technique 3)

# -*- coding: utf-8 -*-
"""
Created on Fri Nov 17 12:42:51 2023

@author: Mav
"""

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Constants
TRAIN_DIR = 'data/train'
VAL_DIR = 'data/test'
EMOTION_DICT = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# Check if the directories exist, create them if not
for directory in [TRAIN_DIR, VAL_DIR]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Data generator
datagen = ImageDataGenerator(rescale=1./255)

# Training and validation generators
train_generator = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    VAL_DIR,
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical'
)

# Model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

# Load trained model
model.load_weights('C:/Users/Mav/Documents/Midterm Exam - Emerging Tech/MD/model.h5')

# Webcam feed
cap = cv2.VideoCapture(0)

while True:
    # Capture frame
    ret, frame = cap.read()
    if not ret:
        break

    # Find haar cascade to draw bounding box around face
    facecasc = cv2.CascadeClassifier('C:/Users/Mav/Documents/Midterm Exam - Emerging Tech/haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facecasc.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        prediction = model.predict(cropped_img)
        maxindex = int(np.argmax(prediction))
        cv2.putText(frame, EMOTION_DICT[maxindex], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame, (500, 500), interpolation=cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#Reference: https://github.com/travistangvh/emotion-detection-in-real-time
#https://github.com/saranshbht/Emotion-detection/blob/master/src/.ipynb_checkpoints/emotions-checkpoint.ipynb

---

## Analysis

For the three different techniques you used in face detection, provide an in-depth analysis.

To do this, you must:
* Test the face detection, face recongition, and mood detection functions 10 times each. Only the mood detection will have components for 10 tests for each different technique used.
* Create a table containing the 10 tests (like shown below) for each task.
* Analyze each output by identifying the accuracy and providing your observations.

In [ ]:
# Import the module for tabulating the data
from tabulate import tabulate

# Create a list for content of the table
test_results = [
    ["1", "1", "YES", 1],
    ["2", "1", "YES", 1],
    ["3", "1", "YES", 1],
    ["4", "1", "YES", 1],
    ["5", "1", "YES", 1],
    ["6", "1", "YES", 1],
    ["7", "1", "YES", 1],
    ["8", "2", "YES", 1],
    ["9", "1", "YES", 1],
    ["10", "2", "YES", 1]

]

# Create a list for the headers of your table
header = ["Test #", "# Recognized Face","Did it meet your expectation?", "Score"]

# display table
print("Task 1: Face Detection")
print(tabulate(test_results, headers=header, tablefmt="grid"))

# Calculate for the accuracy
total = 0
for i in test_results:
    total += i[3]
print("Accuracy: ", round(total/len(test_results)*100,2))

Task 1: Face Detection
+----------+---------------------+---------------------------------+---------+
|   Test # |   # Recognized Face | Did it meet your expectation?   |   Score |
+==========+=====================+=================================+=========+
|        1 |                   1 | YES                             |       1 |
+----------+---------------------+---------------------------------+---------+
|        2 |                   1 | YES                             |       1 |
+----------+---------------------+---------------------------------+---------+
|        3 |                   1 | YES                             |       1 |
+----------+---------------------+---------------------------------+---------+
|        4 |                   1 | YES                             |       1 |
+----------+---------------------+---------------------------------+---------+
|        5 |                   1 | YES                             |       1 |
+----------+-----------------

* Analysis: The face detection was able to detect all of the faces from my camera therefore I considered this very accurate.

In [ ]:
# Import the module for tabulating the data
from tabulate import tabulate

# Create a list for content of the table
test_results = [
    ["1", "2", "1", 0],
    ["2", "2", "2", 1],
    ["3", "2", "2", 1],
    ["4", "2", "2", 1],
    ["5", "2", "2", 1],
    ["6", "2", "2", 1],
    ["7", "2", "2", 1],
    ["8", "2", "2", 1],
    ["9", "2", "2", 1],
    ["10", "2", "2", 1]

]

# Create a list for the headers of your table
header = ["Test #", "Total # of Faces", "# of Faces Recognized", "Score"]

# display table
print("Task 2: Facial Recognition")
print(tabulate(test_results, headers=header, tablefmt="grid"))

# Calculate for the accuracy
total = 0
for i in test_results:
    total += i[3]
print("Accuracy: ", round(total/len(test_results)*100,2))

Task 2: Facial Recognition
+----------+--------------------+-------------------------+---------+
|   Test # |   Total # of Faces |   # of Faces Recognized |   Score |
+==========+====================+=========================+=========+
|        1 |                  2 |                       1 |       0 |
+----------+--------------------+-------------------------+---------+
|        2 |                  2 |                       2 |       1 |
+----------+--------------------+-------------------------+---------+
|        3 |                  2 |                       2 |       1 |
+----------+--------------------+-------------------------+---------+
|        4 |                  2 |                       2 |       1 |
+----------+--------------------+-------------------------+---------+
|        5 |                  2 |                       2 |       1 |
+----------+--------------------+-------------------------+---------+
|        6 |                  2 |                       2 |    

* Analysis: The facial recognition was able to recognize most of the faces from my datasets and it should've been 100% if I showed my whole face on my first trial, therefore I considered this very accurate as well.

In [ ]:
# Import the module for tabulating the data
from tabulate import tabulate

# Create a list for content of the table
test_results = [
    ["1", "Happy", "Happy", 1],
    ["2", "Happy", "Happy", 1],
    ["3", "Happy", "Happy", 1],
    ["4", "Sad", "Sad", 1],
    ["5", "Sad", "Sad", 1],
    ["6", "Sad", "Sad", 1],
    ["7", "Neutral", "Neutral", 1],
    ["8", "Neutral", "Neutral", 1],
    ["9", "Neutral", "Neutral", 1],
    ["10", "Sad", "Happy", 0]

]

# Create a list for the headers of your table
header = ["Test #", "Expected", "Actual", "Score"]

# display table
print("Task 3A: Mood Detection using BRIEF w/ facial recognition and detection")
print(tabulate(test_results, headers=header, tablefmt="grid"))

# Calculate for the accuracy
total = 0
for i in test_results:
    total += i[3]
print("Accuracy: ", round(total/len(test_results)*100,2))

Task 3A: Mood Detection using BRIEF w/ facial recognition and detection
+----------+------------+----------+---------+
|   Test # | Expected   | Actual   |   Score |
+==========+============+==========+=========+
|        1 | Happy      | Happy    |       1 |
+----------+------------+----------+---------+
|        2 | Happy      | Happy    |       1 |
+----------+------------+----------+---------+
|        3 | Happy      | Happy    |       1 |
+----------+------------+----------+---------+
|        4 | Sad        | Sad      |       1 |
+----------+------------+----------+---------+
|        5 | Sad        | Sad      |       1 |
+----------+------------+----------+---------+
|        6 | Sad        | Sad      |       1 |
+----------+------------+----------+---------+
|        7 | Neutral    | Neutral  |       1 |
+----------+------------+----------+---------+
|        8 | Neutral    | Neutral  |       1 |
+----------+------------+----------+---------+
|        9 | Neutral    | Neutral  

* Analysis: This mood detection would be much more accurate if I have more datasets for each mood. I only inputted 20 each mood which isn't enough to cover all of my facial features on different POV.

In [ ]:
# Import the module for tabulating the data
from tabulate import tabulate

# Create a list for content of the table
test_results = [
    ["1", "Happy", "Happy", 1],
    ["2", "Neutral", "Happy", 0],
    ["3", "Happy", "Happy", 1],
    ["4", "Sad", "Sad", 1],
    ["5", "Happy", "Happy", 1],
    ["6", "Happy", "Happy", 1],
    ["7", "Neutral", "Neutral", 1],
    ["8", "Happy", "Happy", 1],
    ["9", "Neutral", "Neutral", 1],
    ["10", "Happy", "Neutral", 0]

]

# Create a list for the headers of your table
header = ["Test #", "Expected", "Actual", "Score"]

# display table
print("Task 3B: Mood detection using Haarcascade.smile")
print(tabulate(test_results, headers=header, tablefmt="grid"))

# Calculate for the accuracy
total = 0
for i in test_results:
    total += i[3]
print("Accuracy: ", round(total/len(test_results)*100,2))

Task 3B: Mood detection using Haarcascade.smile
+----------+------------+----------+---------+
|   Test # | Expected   | Actual   |   Score |
+==========+============+==========+=========+
|        1 | Happy      | Happy    |       1 |
+----------+------------+----------+---------+
|        2 | Neutral    | Happy    |       0 |
+----------+------------+----------+---------+
|        3 | Happy      | Happy    |       1 |
+----------+------------+----------+---------+
|        4 | Sad        | Sad      |       1 |
+----------+------------+----------+---------+
|        5 | Happy      | Happy    |       1 |
+----------+------------+----------+---------+
|        6 | Happy      | Happy    |       1 |
+----------+------------+----------+---------+
|        7 | Neutral    | Neutral  |       1 |
+----------+------------+----------+---------+
|        8 | Happy      | Happy    |       1 |
+----------+------------+----------+---------+
|        9 | Neutral    | Neutral  |       1 |
+----------+

* Analysis: This is less accurate due to the fact that this only detects smiling person and conditionalized to display "Neutral or Sad" when it didn't detect any smile from the camera.

In [ ]:
# Import the module for tabulating the data
from tabulate import tabulate

# Create a list for content of the table
test_results = [
    ["1", "Happy", "Happy", 1],
    ["2", "Angry", "Angry", 1],
    ["3", "Sad", "Neutral", 0],
    ["4", "Surprised", "Surprised", 1],
    ["5", "Surprised", "Surprised", 1],
    ["6", "Happy", "Happy", 1],
    ["7", "Sad", "Angry", 0],
    ["8", "Angry", "Angry", 1],
    ["9", "Sad", "Sad", 1],
    ["10", "Sad", "Angry", 0]

]

# Create a list for the headers of your table
header = ["Test #", "Expected", "Actual", "Score"]

# display table
print("Task 3C: Mood detection using deep convolutional neural networks")
print(tabulate(test_results, headers=header, tablefmt="grid"))

# Calculate for the accuracy
total = 0
for i in test_results:
    total += i[3]
print("Accuracy: ", round(total/len(test_results)*100,2))

Task 3C: Mood detection using deep convolutional neural networks
+----------+------------+-----------+---------+
|   Test # | Expected   | Actual    |   Score |
+==========+============+===========+=========+
|        1 | Happy      | Happy     |       1 |
+----------+------------+-----------+---------+
|        2 | Angry      | Angry     |       1 |
+----------+------------+-----------+---------+
|        3 | Sad        | Neutral   |       0 |
+----------+------------+-----------+---------+
|        4 | Surprised  | Surprised |       1 |
+----------+------------+-----------+---------+
|        5 | Surprised  | Surprised |       1 |
+----------+------------+-----------+---------+
|        6 | Happy      | Happy     |       1 |
+----------+------------+-----------+---------+
|        7 | Sad        | Angry     |       0 |
+----------+------------+-----------+---------+
|        8 | Angry      | Angry     |       1 |
+----------+------------+-----------+---------+
|        9 | Sad       

* Analysis: This is very accurate for various moods although the downside of this it wasn't accurate enough to detect sad emotion. This is much better for detecting multiple moods of a person in a live camera feed without taking a picture of each person and their moods.

---

## Summary and Lessons Learned

I have concluded and learned that environmental factors, objects, and facial features are factors that meddle with the accuracy of facial detection. For example, when using face detection, if you have longer bangs or hair, it doesn't detect your face. Another observation from my trials in facial recognition is that you need to have a high-definition or high-quality raw image for accurate face detection and recognition when it is converted into a grayscale and PGM file.

---

**Proprietary Clause**

Property of the Technological Institute of the Philippines (T.I.P.). No part of the materials made and uploaded in this learning management system by T.I.P. may be copied, photographed, printed, reproduced, shared, transmitted, translated or reduced to any electronic medium or machine-readable form, in whole or in part, without prior consent of T.I.P.

Prepared by Engr. RMR